In [1]:
import os
import numpy as np
import pandas as pd

import keras
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator, load_img
import keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB7

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras import Input, layers, Model, optimizers

from clr_callback import CyclicLR
from tensorflow.keras.callbacks import ModelCheckpoint

from collections import Counter

In [2]:
data_dir = '../input/cassava-leaf-disease-classification/'
train_images_dir = os.path.join(data_dir, 'train_images')
test_images_dir = os.path.join(data_dir, 'test_images')
train_csv = pd.read_csv(os.path.join(data_dir, 'train.csv'), dtype=str)

train_csv, val_csv = train_test_split(train_csv, test_size=0.2, stratify=train_csv.label)

In [3]:
# train_csv.label.value_counts()

In [4]:
batch_size = 8
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory=train_images_dir,
    x_col="image_id",
    y_col="label",
    class_mode='sparse',
    target_size=(256, 256),
    batch_size=batch_size
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_csv,
    directory=train_images_dir,
    x_col="image_id", 
    y_col="label", 
    class_mode='sparse',
    target_size=(256, 256),
    batch_size=batch_size
)

Found 17117 validated image filenames belonging to 5 classes.
Found 4280 validated image filenames belonging to 5 classes.


In [5]:
counter = Counter(train_generator.classes)
total = float(sum(counter.values()))
# max_val = float(max(counter.values()))
class_weight = {class_id : total/(2*num_images) for class_id, num_images in counter.items()}

In [6]:
STEPS_PER_EPOCH = (train_generator.n/train_generator.batch_size)//10

VAL_STEPS_PER_EPOCH = val_generator.n//val_generator.batch_size

epochs=20

In [7]:
def inception_block(input_layer):
    conv1 = Conv2D(64, (1, 1), activation='relu', padding='same')(input_layer)
    # 3x3 conv
    conv3 = Conv2D(32, (1, 1), padding='same', activation='relu')(input_layer)
    conv3 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv3)
    # 5x5 conv
    conv5 = Conv2D(32, (1, 1), padding='same', activation='relu')(input_layer)
    conv5 = Conv2D(64, (5, 5), padding='same', activation='relu')(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_layer)
    pool = Conv2D(32, (1, 1), padding='same', activation='relu')(pool)
    # concatenate filters, assumes filters/channels last
    output_layer = layers.merge.concatenate([conv1, conv3, conv5, pool], axis=-1)
    return output_layer

In [8]:
def residual_block(input_layer):
    conv1 = Conv2D(16, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal')(input_layer)
    conv2 = Conv2D(16, (5, 5), activation='relu', padding='same', kernel_initializer='he_normal')(input_layer)
    conv3 = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    add_layer = layers.add([conv3, conv1])
    output_layer = layers.Activation('relu')(add_layer)
    return output_layer

In [9]:
def custom_loss(y_true, y_pred):
    loss = (-K.log(y_pred)) * y_true
#     loss = np.multiply(loss, y_true)
    loss_weights = [1, 1, 1, 3, 1]
#     loss_weights = [3, 3, 3, 1, 3]
    weighted_loss = loss * loss_weights
    weighted_loss /= sum(loss_weights)
    weighted_loss = K.sum(weighted_loss, axis=1)
    return weighted_loss

In [10]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), padding='same', input_shape=(256, 256, 3), activation='relu'))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2)))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(Conv2D(32, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2,2)))
# model.add(Flatten())
# model.add(Dense(10, activation='relu'))
# model.add(Dense(5, activation='softmax'))

# model.compile(keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=["accuracy"])
# history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=100)

In [11]:
# dropout_rate=0.3
# input = Input(shape=(300,400,3))
# batch_norm = layers.BatchNormalization()(input)
# inception1 = inception_block(input)
# o1 = Dropout(dropout_rate)(inception1)
# residual1 = residual_block(o1)
# o2 = Dropout(dropout_rate)(residual1)
# # inception2 = inception_block(o2)
# flatten = Flatten()(o2)
# dense1 = Dense(32, activation='relu')(flatten)
# output = Dense(5, activation='softmax')(dense1)
# model2 = Model(input, output)
# model2.summary()


In [12]:
# model2.compile(loss=custom_loss,
#                    optimizer=optimizers.Adam(lr=1e-8),
#                    metrics=['accuracy'])

# STEPS_PER_EPOCH = train_generator.n//train_generator.batch_size
# # STEPS_PER_EPOCH = 200
# epochs=50
# history = model2.fit_generator(train_generator, steps_per_epoch=STEPS_PER_EPOCH, epochs=epochs, 
#                                validation_data=val_generator, validation_steps=50, use_multiprocessing=True)
# # history = model2.fit(train_generator, steps_per_epoch=STEPS_PER_EPOCH, epochs=epochs, validation_data=val_generator, validation_steps=50)

In [13]:
# step_size = 4 * (train_generator.n // batch_size)
step_size = 8 * STEPS_PER_EPOCH * batch_size 
cycliclr_config = {
    'step_size': step_size,
    'mode': "exp_range",
    'gamma': 0.9,
    'base_lr': 1e-6,
    'max_lr': 1e-3,
}
clr = CyclicLR(**cycliclr_config)

In [14]:
checkpoint_filepath = "new-weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor="val_accuracy",
    verbose=1, 
    save_best_only=True
    )

In [15]:
dropout_rate = 0.2

base = EfficientNetB7(
    include_top = False,
    weights = None, 
    input_shape = (256, 256, 3)
)
pool = layers.GlobalMaxPool2D()(base.output)
flatten = Flatten()(pool)
flatten = layers.BatchNormalization()(flatten)
dense1 = Dense(512, activation='relu')(flatten)
dense1 = Dropout(dropout_rate)(dense1)
dense2 = Dense(512, activation='relu')(dense1)
dense2 = Dropout(dropout_rate)(dense2)
output = Dense(5, activation='softmax')(dense2)
model = Model(base.inputs, output)

trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

Total params: 65,684,380.0
Trainable params: 65,368,533
Non-trainable params: 315,847.0


In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy', 
#     loss=custom_loss,
    optimizer=optimizers.RMSprop(lr=cycliclr_config['base_lr'], momentum=0.8),
    metrics=['accuracy']
)

history = model.fit_generator(train_generator, steps_per_epoch=STEPS_PER_EPOCH, epochs=epochs, 
                              validation_data=val_generator, validation_steps=VAL_STEPS_PER_EPOCH, use_multiprocessing=True, 
                              shuffle=True, callbacks=[clr, checkpoint_callback], class_weight=class_weight, verbose=1)

Epoch 1/20
213/213 [==============================] - ETA: 0s - loss: 4.5670 - accuracy: 0.2324
Epoch 00001: val_accuracy improved from -inf to 0.10234, saving model to new-weights-improvement-01-0.10.hdf5
213/213 [==============================] - 178s 838ms/step - loss: 4.5670 - accuracy: 0.2324 - val_loss: 1.7785 - val_accuracy: 0.1023
Epoch 2/20
178/213 [========================>.....] - ETA: 16s - loss: 4.3850 - accuracy: 0.2381

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [ ]:
pred_labels = model.predict(val_generator)
pred_labels = np.argmax(pred_labels, axis=1)

In [ ]:
cm = confusion_matrix(np.asarray(val_csv.label, dtype='int'), pred_labels)
classes = [0, 1, 2, 3, 4]
plot_confusion_matrix(cm, classes)

In [ ]:
img = load_img(os.path.join(train_images_dir, train_csv.iloc[0][0]))
img = np.asarray(img)

In [ ]:
# cv2.extractChannel(img)
# r, g, b = cv2.split(np.asarray(img))

In [ ]:
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
plt.imshow(cv2.extractChannel(img, 2))
plt.show()